# 30-Day Scheduler Sync (Refactored)

This notebook syncs event data from a Google Sheet to a Google Calendar. It leverage centralized utilities for consistency.

## Step 1: Setup and Authentication

In [1]:
import os
import sys
import pandas as pd
import datetime

# Add src to path so we can import our modules
sys.path.append('src')

from utils_calendar_general import get_google_services, create_calendar_event
from sync_scheduler import prepare_event_body

# --- Configuration ---
SPREADSHEET_ID = '10Z993MrZHH0Da_pXEFZoo0MBdxKhf619fZSuuvaAdlQ'
SIGNUP_SHEET = 'Signup'
CONTACT_SHEET = 'Teacher Contact'
CALENDAR_ID = 'b45a2d5121fed950d815cfa167dd4b3a6aa74c5d62fea928702e6f4300d96545@group.calendar.google.com'
TEMPLATE_FILE = '_calendar_event_template.jsonc'
CREATED_EVENTS_CSV = 'logs/created_events.csv'
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets.readonly', 
    'https://www.googleapis.com/auth/calendar.events'
]

# Credentials Paths
TOKEN_PATH = '.credentials/token.json'
CREDS_PATH = '.credentials/credentials.json'

sheets_service, calendar_service = get_google_services(TOKEN_PATH, CREDS_PATH, SCOPES)
print("Services initialized.")

Services initialized.


## Step 2: Fetch Event Data from Google Sheets

In [2]:
from google_sheets_data import fetch_google_sheets_data

# Fetch data using the external module
df_pending, teacher_map = fetch_google_sheets_data(
    sheets_service=sheets_service,
    spreadsheet_id=SPREADSHEET_ID,
    signup_sheet=SIGNUP_SHEET,
    contact_sheet=CONTACT_SHEET,
    template_file=TEMPLATE_FILE
)
display(df_pending.head())

Loaded 14 teacher contacts.
Found 54 events in Google Sheets.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration
0,45-Minute Guided Session: Stephen Holsenbeck,2026-02-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-01,Sunday,07:00,07:45,45
1,45-Minute Guided Session: Stephen Holsenbeck,2026-02-02T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-02,Monday,07:00,07:45,45
2,45-Minute Guided Session: Stephen Holsenbeck,2026-02-03T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-03,Tuesday,07:00,07:45,45
3,10-Minute Guided Session: Eileen Knott,2026-02-03T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-02-03,Tuesday,20:00,20:10,10
4,10-Minute Guided Session: Rob Thompson,2026-02-03T21:00:00,Rob Thompson,"{'First Name': 'Rob', 'Last Name': 'Thompson',...",2026-02-03,Tuesday,21:00,21:10,10


## Step 3: Overlap Detection

In [3]:
from overlap_detection import check_overlaps, update_created_events_csv

UPDATE_CREATED_EVENTS = True # Set to True to refresh from Calendar

if UPDATE_CREATED_EVENTS:
    update_created_events_csv(calendar_service, CALENDAR_ID, CREATED_EVENTS_CSV)

if os.path.exists(CREATED_EVENTS_CSV):
    df_created = pd.read_csv(CREATED_EVENTS_CSV)
    print(f"Loaded {len(df_created)} already created events.")
    
    # Mark overlaps
    df_pending['is_overlap'] = check_overlaps(df_pending, df_created)
    
    overlaps = df_pending[df_pending['is_overlap'] == True]
    print(f"Detected {len(overlaps)} overlapping events.")
    display(overlaps)
else:
    df_pending['is_overlap'] = False
    print("No created_events.csv found. Starting fresh.")

Syncing logs/created_events.csv with Google Calendar...
Successfully synced 186 events to logs/created_events.csv.
Loaded 186 already created events.
Detected 0 overlapping events.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration,is_overlap


## Step 4: User Review and Pruning

In [4]:
print("Review the overlaps above.")
confirm = input("Remove overlapping events from the list to be created? (y/n): ")

if confirm.lower() == 'y':
    df_to_create = df_pending[df_pending['is_overlap'] == False].copy()
    print(f"Pruned {len(df_pending) - len(df_to_create)} events. {len(df_to_create)} remaining.")
else:
    df_to_create = df_pending.copy()
    print(f"Proceeding with all {len(df_to_create)} events.")
display(df_to_create.head())

Review the overlaps above.
Proceeding with all 54 events.


,Summary,Begin,Teacher,Contact,Date,Day,Start,End,Duration,is_overlap
0,45-Minute Guided Session: Stephen Holsenbeck,2026-02-01T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-01,Sunday,07:00,07:45,45,False
1,45-Minute Guided Session: Stephen Holsenbeck,2026-02-02T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-02,Monday,07:00,07:45,45,False
2,45-Minute Guided Session: Stephen Holsenbeck,2026-02-03T07:00:00,Stephen Holsenbeck,"{'First Name': 'Stephen', 'Last Name': 'Holsen...",2026-02-03,Tuesday,07:00,07:45,45,False
3,10-Minute Guided Session: Eileen Knott,2026-02-03T20:00:00,Eileen Knott,"{'First Name': 'Eileen', 'Last Name': 'Knott',...",2026-02-03,Tuesday,20:00,20:10,10,False
4,10-Minute Guided Session: Rob Thompson,2026-02-03T21:00:00,Rob Thompson,"{'First Name': 'Rob', 'Last Name': 'Thompson',...",2026-02-03,Tuesday,21:00,21:10,10,False


## Step 5: Preview and Create Events

In [ ]:
template_text = open(TEMPLATE_FILE, 'r').read()
print(f"Events to create: {len(df_to_create)}")

run_confirm = input("Proceed with event creation? (y/n): ")

if run_confirm.lower() == 'y':
    for idx, row in df_to_create.iterrows():
        try:
            event_body = prepare_event_body(row, template_text)
            created_event = create_calendar_event(calendar_service, CALENDAR_ID, event_body)
            
            # Incremental save
            new_row = pd.DataFrame([{
                'Summary': created_event.get('summary'),
                'ID': created_event.get('id'),
                'Begin': created_event['start'].get('dateTime') or created_event['start'].get('date')
            }])
            
            if os.path.exists(CREATED_EVENTS_CSV):
                new_row.to_csv(CREATED_EVENTS_CSV, mode='a', header=False, index=False)
            else:
                new_row.to_csv(CREATED_EVENTS_CSV, index=False)
            
            print(f"Created: {created_event.get('summary')} ({created_event.get('id')})")
        except Exception as e:
            print(f"Failed to create event for {row['Summary']}: {e}")
else:
    print("Creation cancelled.")

## Step 6: Delete Events (Cleanup)

In [ ]:
from utils_calendar_general import delete_events_from_csv

# To delete events, uncomment and run the line below:
# delete_events_from_csv(CREATED_EVENTS_CSV, calendar_service, CALENDAR_ID)